In [2]:
import os
import json
from time import time
from bitarray import bitarray

In [24]:
arr:bytes
t0 = time()
with open("/root/code/outtest/preprocessed/state/vector/upgrade.json", "r") as j:
    dic = json.load(j)
    arr_shape = dic["upgrades_array_shape"]
    t1 = time()
    print(f"load_json: {t1-t0}")
with open("/root/code/outtest/preprocessed/state/vector/upgrade.bin", "rb") as f:
    t2 = time()
    print(f"open_bin: {t2-t1}")
    arr = f.read()
    t3 = time()
    print(f"read_bin: {t3-t2}")
    np_arr = decode_to_numpy(arr, arr_shape[0]*arr_shape[1], 1)
    t4 = time()
    print(f"decode_bin: {t4-t3}")
np_arr = np_arr.reshape(arr_shape)
print(np_arr.shape)

load_json: 0.0004379749298095703
open_bin: 0.00039458274841308594
read_bin: 7.224082946777344e-05
decode_bin: 4.976996183395386
(9817, 320)


In [5]:
root_path = "/root/code/outtest/preprocessed/state"
outdict = {}

In [28]:

with open(os.path.join(root_path, f"world/bits_info.json"), "r") as j:
    bits_info = json.load(j)
for feature in os.listdir(os.path.join(root_path, "world")):
    if (feature.split(".")[-1] != "bin"):
        continue
    with open(os.path.join(root_path, f"world/{feature}"), "rb") as f:
        byte_file = f.read()
        decode_to_numpy(byte_file, (128*128*2915), bits_info[feature.split(".")[0]])
        print(feature.split(".")[0], len(byte_file)*8/(128*128*2915))
        #outdict[feature.split(".")[0]] = np.frombuffer(f.read()).reshape((128, 128,-1))
       

pathable 1.0
buildable 1.0
creep 1.0


KeyboardInterrupt: 

In [8]:
outdict["height_map"].shape

array([5.96420884e-212, 5.96430896e-212, 1.60870561e-211, ...,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000])

In [10]:
from bitarray import bitarray

# 비트 단위로 파일 작성
bits = bitarray('1101010110110001')
with open('output.bin', 'wb') as file:
    bits.tofile(file)

# 비트 단위로 파일 읽기
read_bits = bitarray()
with open(os.path.join(root_path, "world/visibility_map.bin"), 'rb') as file:
    read_bits.fromfile(file)

print(type(read_bits))


arr = np.zeros(int(len(read_bits)/8), dtype=np.uint8)
for i, bit in enumerate(read_bits):
    arr[i//8] += bit << (i%8)
    



<class 'bitarray.bitarray'>


In [11]:
np.max(arr)

128

In [2]:
import os
import json
import numpy as np
root_path = "/mount/jyj/preprocessed/2019_WCS_Spring/307e3655dee84b0d93b14685baf68355/0.bin"
# open details.json\

def decode_to_numpy(byte_array:bytes, n_items:int, bits_per_elem, buffer_type:type=np.uint8()):
    """decode compact bit information to numpy array

    Args:
        byte_array (bytes): byte array to change
        bits_per_elem (int): 
        n_items (int): number of items that decoded array has
        buffer_type (type, optional): type of buffer, recommanded to be same with encode function. Defaults to np.uint8.

    Returns:
        _type_: 1 demensional numpy array
    """
    if (bits_per_elem==1):
        elem_type = bool
        
    elif(bits_per_elem==8):
        elem_type = np.uint8
    else:
        assert False, "only bool, np.uint8 available"
    bits_per_buffer = 8*buffer_type.itemsize
    read_arr = np.frombuffer(byte_array, dtype=buffer_type)
    decoded_arr = np.zeros(n_items, dtype=elem_type)
    #print(read_arr)
    for idx in range(n_items):
        decoded_arr[idx] = (read_arr[int(idx/(bits_per_buffer*bits_per_elem))] >> ((bits_per_buffer-1) - (idx%bits_per_buffer)))%(2**bits_per_elem)

    
    return decoded_arr



class Decoder:
    """Decoder class
    """
    def __init__(self, root_path):
        """__init__

        Args:
            root_path (str): path of a match
        """
        # open detail json file and get detail
        self.info_dict : dict
        self.height_map = None
        
        with open(os.path.join(root_path, "details.json"), "r") as j:
            self.info_dict = json.load(j)
        

    
        
    def decode_world(self, bin_arr):
        """decode world minimap feature

        Args:
            bin_arr (bytes): byte information of world feature

        Returns:
            nmupy.array: 128*128*8 array
            index of last axis: ["height_map", "visibility_map", "creep", "player_relative", "alerts", "pathable", "buildable", "camera"]
        """
        # 바꿀꺼면 바꾸셈
        map_size = self.info_dict["world"]["size"]
        outarr = np.ones((map_size[0], map_size[1], 8), dtype=np.uint8)
        bits_info = self.info_dict["world"]["bits_per_pixel"]

        
        # decode height map if needed
        if (self.height_map is None):
            with open(self.info_dict["world"]["height_map"], "rb") as f:   
                 
                self.height_map = decode_to_numpy(f.read(), map_size[0]*map_size[1], bits_info["height_map"])  
        outarr[:,:,0] = self.height_map.reshape((map_size[0], map_size[1]))
        
        # decode other informations
        start_index = 0
        for idx, key in enumerate(bits_info.keys()):
            
            if (key == "height_map"):
                continue
            end_index = start_index + map_size[0]*map_size[1]*bits_info[key]//8
            
            outarr[:,:,idx] = decode_to_numpy(bin_arr[start_index:end_index], map_size[0]*map_size[1], bits_info[key]).reshape(map_size)
            start_index = end_index
        return outarr
    
    def decode_vector(self, bin_arr):
        
        outarr= np.frombuffer(bin_arr[:28], dtype=np.int16)
        upgrades = decode_to_numpy(bin_arr[28:], 320, 1)
        return outarr, upgrades
    
    def decode_units(self, bin_arr):
        outarr = np.frombuffer(bin_arr, dtype=np.int16)
        return outarr
    def decode(self, bin_dir):
        """decode encoded featrue binary file

        Args:
            bin_dir (str): path of the bin file

        Returns:
            dict: {"world": world, "vector": vector, "upgrades": upgrades, "units": units}
            world: 128 * 128 * 8 nnumpy array (np.uint8)
            vector: (14, ) numpy array (np.int16)
            upgrades: (320, ) one hot encoded numpy array (bool)
            units: (46, n_units) numpy array (np.int16)
        """
        n_world_bits = 1
        map_size = self.info_dict["world"]["size"]
        world_bits = 0
        for i in self.info_dict["world"]["bits_per_pixel"].keys():
            if (i == "height_map"):
                continue
            world_bits += self.info_dict["world"]["bits_per_pixel"][i]
        
        n_world_bits = (map_size[0] * map_size[1]*world_bits)
        
        n_vector_bits = 16 * 14 + 320 + n_world_bits
        
        with open(bin_dir, "rb") as b:
            tot_arr = b.read()
            world_flatten = self.decode_world(tot_arr[:n_world_bits//8])
            vector, upgrades = self.decode_vector(tot_arr[n_world_bits//8:n_vector_bits//8])
            units_flatten = self.decode_units(tot_arr[n_vector_bits//8:])
        world = world_flatten.reshape((map_size[0], map_size[1], -1))
        units = units_flatten.reshape((46, -1))
        return {"world": world, "vector": vector, "upgrades": upgrades, "units": units}


In [3]:
decoder = Decoder("/mount/jyj/preprocessed/2019_WCS_Spring/307e3655dee84b0d93b14685baf68355/player1")
a = decoder.decode("/mount/jyj/preprocessed/2019_WCS_Spring/307e3655dee84b0d93b14685baf68355/player1/01982.bin")

In [4]:
lst = [0]
for i in range(8):
    for row in a["world"][:,:,i]:
        for col in row:
            if col != 0 and i != 0:
                if (lst[-1] != i):
                    lst.append(i)
                    
                    

print(lst)

[0, 2, 5, 6, 7]


In [107]:
import os
path = "/mount/jyj/replays/"
path_pre = "/mount/jyj/preprocessed/"
for i in os.listdir(path):
    for j in os.listdir(os.path.join(path, i)):
        for k in range(2):
            
            os.makedirs(os.path.join(path_pre, i, j.split(".")[0], f"player{k+1}"), exist_ok=True)


In [1]:
import os
command = "python3 /root/code/mypysc2/bin/extract_state.py --replay /mount/jyj/replays/2019_WCS_Spring/307e3655dee84b0d93b14685baf68355.SC2Replay"

data_root_path = "/mount/jyj/replays/2019_WCS_Spring"
out_root_path = "/mount/jyj/preprocessed/2019_WCS_Spring/"
prog = []
done = []
yet = []
for game in os.listdir(data_root_path):
    
    for i in (1, 2):
        save_dir = game.split(".")[0] + f"/player{i}"
        if (len(os.listdir(os.path.join(out_root_path, save_dir))) == 1):
            #print(os.listdir(os.path.join(out_root_path, save_dir, f"player{i}")))
            prog.append(save_dir)
            
        elif (len(os.listdir(os.path.join(out_root_path, save_dir))) > 1):
            done.append(save_dir)
        else:
            yet.append(save_dir)


print(len(prog))
print(len(done))
print(len(yet))

325
753
4


In [18]:
with open("/mount/jyj/preprocessed/2019_WCS_Spring/307e3655dee84b0d93b14685baf68355/player1/0001.bin", "rb") as f:
    a = f.read()
    print(len(a))
    n = 0
    print(a[460938])
            

461399
0


In [24]:
(128*128*28+14*16+320)/10**3


459.296

In [117]:
from time import time

t0 = time()
for i in range(100000):
    a = 2**i
print(time() - t0)
t1 = time()
for j in range(100000):
    a = 1 << i
print(time() - t1)

7.515561819076538
0.02631855010986328


In [4]:
import os
import json
from tqdm import tqdm
root_path = "/mount/jyj/unpacked/2019_WCS_Spring/2019_WCS_Spring_data"
outdict = {}
for match in tqdm(os.listdir(root_path)):
    with open(os.path.join(root_path, match), "r") as j:
        json_dict = json.load(j)
        for user in json_dict["ToonPlayerDescMap"].keys():
            if (json_dict["ToonPlayerDescMap"][user]["race"] != "Prot"):
                continue
            try:
                outdict[json_dict["metadata"]["mapName"]].append(match.split(".")[0])
            except:
                outdict[json_dict["metadata"]["mapName"]] = [match.split(".")[0]]
            
    
max_map = ""
max_matches = 0    
for map_name in outdict.keys():
    if len(outdict[map_name]) > max_matches:
        max_map = map_name
        max_matches = len(outdict[map_name])
        
print(max_map, max_matches)

100%|██████████| 540/540 [00:35<00:00, 15.29it/s]

Port Aleksander LE 76


In [2]:
print(outdict)

{}


In [75]:
import os
import json
root_path = "/mount/jyj/preprocessed/2019_WCS_Spring/"

def get_matchIDs(base_path, race = ("Protoss", "Protoss"), map = -1):
    """get matchIDs support the following race and map condition

    Args:
        base_path (str): path of the competition
        race (tuple or -1, optional): 2 element tuple, each corresponds to race of p1, p2. 
                                If element is -1, every race is available Defaults to ("Protoss", "Protoss").
                                e.g. -1: returns every match id (not tuple this case)
                                     (*, -1): Don't care player2's race
        map (str or -1, optional): _description_. Defaults to -1.
    """
    outlist = []
    
    for matchID in os.listdir(base_path):
        p1_flag = 0
        p2_flag = 0
        # get all matches
        if (race == -1 and map == -1):
            outlist.append(matchID)
            continue
        # get meta.json if exist
        try:
            
            with open(os.path.join(base_path, matchID, "player1/state/meta.json"), "r") as j: # player1/meta.json also has player2 information
                meta = json.load(j)
                
        except:
            continue
        
        # check if map is valid
        if (map != -1 and meta["mapName"] != map):
            continue
        
        
        # get all matches if map is same
        if (race == -1):
            outlist.append(matchID)
            continue
        
        for player_info in meta["playerInfo"]: # for each player check the race
            
            player_id = int(player_info["playerInfo"]["playerId"])
            
            if (player_id == 1 and (race[0] == -1 or race[0] == player_info["playerInfo"]["raceActual"])):
                p1_flag = 1
            elif (player_id == 2 and (race[1] == -1 or race[1] == player_info["playerInfo"]["raceActual"])):
                p2_flag = 1
            
            elif (player_id not in (1, 2)):
                assert False, f"PlayerID error: {player_id}"
        if (p1_flag and p2_flag): # if races satisfy the condition
            outlist.append(matchID)
        
    return outlist
    

In [76]:
wanted = get_matchIDs(root_path, race = ("Zerg", "Terran"), map = -1)
#wanted = get_matchIDs(root_path, race = (-1), map = -1)
print(wanted)

['2498dfe6db7f4458b30f62b8c79beca3', '33c547f3eb6c4a6cb3f3c42fc8033821', '278b977530d444108cde0d3cec547f7b', '42245d2a223c4b42ba5fd5f6c11a4b39', '618495a1e10341a8b0731025f1a33ca1', '55a264fd2e164c58842383908c861d7d', '13f7d01643fa4a1ca1235d1a758e065d', '527a7c838d15417782b43c2178954b0a', '7629481c3d624b15b296f646dae67af6', '1c37349530cc44d398f10b56182f64fd', '2e6139ca3ed74294ba8f26e4ceb56bba', '6a4108891059421ba15cb3bfe3c9fb6f', '0a0db23531e2472e9fdb44fed14dd3a4', '2125ad5541e84eeb855a89f8fef8b355', '104cd9a23ffb4ed59469faf67b756921', '6fe1b2a4b15644ccbd9f4d57178a5aaf', '126d0c44bd384b76a9aee8e26f4d9a47', '0ec0c1b09b1344b2ab54cd0e453be260', '4cc8569ef6bb45b1b2dedeea7aedcce7', '687d148b448c403c8b2801ec7cff87b8', '363c24ffe1a9459ba8520ea52dfb29a3', '17b7353563ba466480bc32207286154b', '3b8d792efd2c4371b1e2237fc0411313', '49370b6d937b44058d04e5157f172317', '0613d343aa784b44a9054b027467b02b', '3ed889b93e4c42118bbad08768b2337b', '0186850fd6c64782b23fcc7805cbf3ce']


In [2]:
import os
base_dir = "/mount/StarCraftII/"

for i in os.listdir(base_dir):
    if (i=="Versions"):
        continue
    dir2move = os.path.join(base_dir, i, "Versions", f"Base{i}")
    os.system(f"cp -r {dir2move} /root/StarCraftII/Versions")
    

cp: cannot stat '/mount/StarCraftII/.build.info/Versions/Base.build.info': Not a directory
cp: cannot stat '/mount/StarCraftII/59587/Versions/Base59587': No such file or directory
